# 数据处理专用

In [1]:
import cudf

In [3]:
cudf.read_csv('/data/DataSets/BW_INDEX_CSMAR/BW_5_CSMAR/QX_DIVIDENDPREMIUM.csv')

,﻿SgnYear,Dnum,NDnum,DMB,NDMB,PDND,LogPDND
0,1991,1,14,0.6049,4.8106,-4.2057,-0.9005
1,1992,4,72,2.4407,4.5652,-2.1245,-0.2720
2,1993,42,182,2.2759,2.6085,-0.3325,-0.0592
3,1994,160,193,1.3538,1.7117,-0.3578,-0.1019
4,1995,265,116,1.0813,1.1420,-0.0607,-0.0237
5,1996,239,357,1.8867,2.1904,-0.3038,-0.0648
6,1997,215,599,2.1294,2.4667,-0.3373,-0.0639
7,1998,283,638,2.0322,2.2689,-0.2367,-0.0479
8,1999,304,719,2.2116,2.4395,-0.2278,-0.0426
9,2000,353,818,2.9219,3.6668,-0.7449,-0.0986
